# **Code Example**

In [ ]:
!pip install azure-cosmos


In [ ]:
import uuid
import time
from azure.cosmos import CosmosClient, PartitionKey, exceptions
from statistics import mean


COSMOS_URI = "<my-uri>"
COSMOS_KEY = "<my-key>"
DATABASE_NAME = "ecommerce-db"
CONTAINER_NAME = "eCommerce"

# Setup client and database
client = CosmosClient(COSMOS_URI, COSMOS_KEY)
db = client.create_database_if_not_exists(DATABASE_NAME)
container = db.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/partitionKey"),
    offer_throughput=400
)

# Helper functions
def create_product():
    item = {
        "id": str(uuid.uuid4()),
        "type": "product",
        "name": "Laptop",
        "price": 999.99,
        "partitionKey": "product"
    }
    return container.upsert_item(item)

def read_products():
    start = time.time()
    items = list(container.query_items(
        query="SELECT * FROM c WHERE c.type='product'",
        enable_cross_partition_query=True
    ))
    duration = (time.time() - start) * 1000  # in ms
    return duration, items

def create_inventory(product_id):
    item = {
        "id": str(uuid.uuid4()),
        "type": "inventory",
        "productId": product_id,
        "stockLevel": 50,
        "partitionKey": "inventory"
    }
    return container.upsert_item(item)

def update_inventory(inventory_id):
    try:
        item = container.read_item(item=inventory_id, partition_key="inventory")
        item["stockLevel"] -= 1
        return container.replace_item(item=inventory_id, body=item)
    except exceptions.CosmosResourceNotFoundError:
        print("Inventory item not found.")
        return None

# Experiment Simulation
def run_simulation():
    latencies = []

    print("Creating a product...")
    product = create_product()

    print("Creating inventory...")
    inventory = create_inventory(product_id=product['id'])

    print("Running reads...")
    for _ in range(5):
        latency, _ = read_products()
        latencies.append(latency)

    print("Updating inventory...")
    update_inventory(inventory_id=inventory['id'])

    print("Average Read Latency:", round(mean(latencies), 2), "ms")

# Run
if __name__ == "__main__":
    run_simulation()


# **Sample Output Shape**

In [ ]:
Creating a product...
Product created with ID: f89b1d68-58f3-4f75-b128-9c75a68e60ab

Creating inventory...
Inventory created with ID: 38afac9e-37b2-4b28-8f3c-bbcbab7e233e

Running reads...
Read #1 latency: 14.23 ms
Read #2 latency: 13.56 ms
Read #3 latency: 14.78 ms
Read #4 latency: 15.02 ms
Read #5 latency: 13.99 ms

Updating inventory...
Inventory stock level decremented. New stock level: 49

Average Read Latency: 14.32 ms